#

In [125]:
#!pip install shapely

In [126]:
#!pip install datetime

# **Importing Required Libraries**

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import Output, Input, Dash
import dash_bootstrap_components as dbc
import plotly.express as px
from urllib.request import urlopen
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import json
import shapely
from shapely.geometry import Point, Polygon
from dash import State
import re
import datetime
from datetime import date, datetime, timedelta
import requests

In [9]:
masterDf = pd.read_csv('CMPD_Incidents2.csv')

C:\Users\Owner\AppData\Local\Temp\ipykernel_26444\3129091874.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  masterDf = pd.read_csv('CMPD_Incidents2.csv')


In [10]:
#Checking amount of rows and columns in Dataframe

masterDf.head(1)

,X,Y,YEAR,INCIDENT_REPORT_ID,LOCATION,CITY,STATE,ZIP,X_COORD_PUBLIC,Y_COORD_PUBLIC,...,LOCATION_TYPE_DESCRIPTION,PLACE_TYPE_DESCRIPTION,PLACE_DETAIL_DESCRIPTION,CLEARANCE_STATUS,CLEARANCE_DETAIL_STATUS,CLEARANCE_DATE,HIGHEST_NIBRS_CODE,HIGHEST_NIBRS_DESCRIPTION,OBJECTID,GlobalID
0,1.471035e+06,555315.000127,2023,20230610-1356-01,5700 N TRYON ST,CHARLOTTE,NC,28213,1471035,555315,...,Outdoors,Commercial Place,Other - Commercial Place,Open,Open,NaN,240,Motor Vehicle Theft,1,{277CD4BD-6F2A-49D3-8351-6E020F77AE22}


In [11]:
# Dropping Unessesary Columns 

DropColumns = ['X', 'Y', 'X_COORD_PUBLIC', 'Y_COORD_PUBLIC', 'CLEARANCE_DETAIL_STATUS', 
               'GlobalID', 'CITY', 'STATE','ADDRESS_DESCRIPTION', 'LOCATION_TYPE_DESCRIPTION']
masterDf.drop(columns = DropColumns, inplace = True)

In [12]:
#Cleaning Data so that I can convert the fields to Datatime datatype

masterDf['CLEARANCE_DATE'] = masterDf['CLEARANCE_DATE'].fillna('nan').astype(str)
masterDf['DATE_INCIDENT_END'] = masterDf['DATE_INCIDENT_END'].fillna('nan').astype(str)

def clean_text(text):
    return text.split(' ', 1)[0] #Defining function to take only the dates and leave the times

dateColumns = ['DATE_REPORTED', 'DATE_INCIDENT_BEGAN', 'DATE_INCIDENT_END','CLEARANCE_DATE']
masterDf[dateColumns] = masterDf[dateColumns].apply(lambda x: x.apply(clean_text))

masterDf[dateColumns].head()

,DATE_REPORTED,DATE_INCIDENT_BEGAN,DATE_INCIDENT_END,CLEARANCE_DATE
0,2023/06/10,2023/06/10,2023/06/10,nan
1,2023/06/15,2023/06/15,nan,nan
2,2021/10/01,2021/10/01,2021/10/01,nan
3,2019/08/29,2019/08/19,2019/08/19,nan
4,2022/12/05,2022/11/20,2022/11/20,nan


In [13]:

#Used first for loop to ensure pandas could easily parse the dates, second for loop converts all date columns to datetime64 datatype.
for column in dateColumns:
    masterDf[column] = masterDf[column].str.replace('/', '-')
for column in dateColumns:
    masterDf[column] = pd.to_datetime(masterDf[column], errors = 'coerce')

#ALl date columns have been convered to datetime64
print(masterDf[dateColumns].info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654489 entries, 0 to 654488
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   DATE_REPORTED        654489 non-null  datetime64[ns]
 1   DATE_INCIDENT_BEGAN  654433 non-null  datetime64[ns]
 2   DATE_INCIDENT_END    492793 non-null  datetime64[ns]
 3   CLEARANCE_DATE       264798 non-null  datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 20.0 MB
None


#### The metadata states that NIBRS codes in 8XXs are not considered criminal, so they will be removed from this dataframe. Metadata also states that a Clearance Status of 'Unfounded' means the report was either a false report or that no criminal activity was found, so we will drop these as well


In [14]:
#Identifying what the 8XX codes are
codeDf = masterDf.HIGHEST_NIBRS_CODE.unique()
falseCodes = []
for code in codeDf:
    if (code[0] == '8'):
        falseCodes.append(code)

#Dropping entries where CLEARANCE_STATUS is Unfounded
masterDf = masterDf[masterDf['CLEARANCE_STATUS'] != 'Unfounded']
masterDf['CLEARANCE_STATUS'].value_counts() #Checking to see that 'Unfounded' is not longer in the DataFrame

#Excluding entries in dataframe where the value for 'HIGHEST_NIBRS_CODE' are found in falseCodes (8XX NIBRS codes)
masterDf = masterDf[~masterDf['HIGHEST_NIBRS_CODE'].isin(falseCodes)]
masterDf['HIGHEST_NIBRS_CODE'].unique() #Checking to see if 8XX values are found in 'HIGHEST_NIBRS_CODE' column

array(['240', '13B', '23F', '290', '23C', '520', '26B', '26C', '90Z',
       '35A', '23H', '13A', '100', '120', '220', '35B', '23G', '11D',
       '26A', '280', '23D', '13C', '26F', '90B', '90D', '11A', '26G',
       '200', '270', '26E', '99Y', '250', '370', '23B', '99Z', '36A',
       '90J', '210', '09A', '11B', '720', '36B', '23A', '09C', '90C',
       '90F', '90H', '40A', '90G', '90A', '09B', '64A', '23E', '11C',
       '64B', '39C', '40B', '26D', '39A', '39B', '510', '40C'],
      dtype=object)

# **ZIP CODE WORK**

#### Filtering Dataframe for Charlotte Zip Codes

In [15]:
clt_zip_codes = ['28202', '28203', '28204', '28205', '28206', '28207', '28208', '28209', '28210', '28211', '28212', '28213',
                 '28214', '28215', '28216', '28217', '28226', '28227', '28244', '28246', '28254', '28262', '28269', '28270',
                 '28273', '28277', '28278', '28280', '28281', '28282', '28284', '28285', '28287', '28105', '28134', '28031', 
                 '28036', '28078']

print('Amount of entries before cleaning:', masterDf.shape[0])
print('Number of unique zip codes before cleaning:', len(dict(masterDf['ZIP'].value_counts())))
print('\n')

masterDf['ZIP'] = masterDf['ZIP'].apply(lambda x: re.sub(r'\D', '', str(x)))

#After December of 2019, the department started entering zipcodes in the format '28223.0' as opposed to '28223'. I want these to be treated the same for analysis purposes
#Converting zipcodes to a string and taking the first 5 elements
masterDf['ZIP'] = masterDf['ZIP'].astype(str).str[:5]

# Convert back to integers
masterDf['ZIP'] = masterDf['ZIP'].astype(int, errors = 'ignore')

#First condition filters for charlotte zip codes, second filter includes null values that will be filled later, third filter keeps values that start with '282'
#and have a length less than 5, some zip code values in the dataframe are like '2828' or '2821', these zip codes may be charlotte zip codes but we will have to set them
#to null and find out what they truly are in the next few functions.
masterDf = masterDf[(masterDf['ZIP'].isin(clt_zip_codes)) | (masterDf['ZIP']=='') | ((masterDf['ZIP'].str.startswith('282')) & (masterDf['ZIP'].str.len() < 5))]

print('Amount of entries after cleaning:', masterDf.shape[0])
print('Number of unique zip codes after cleaning:', len(dict(masterDf['ZIP'].value_counts())))


Amount of entries before cleaning: 539334
Number of unique zip codes before cleaning: 822


Amount of entries after cleaning: 536490
Number of unique zip codes after cleaning: 49


Filling null values and zip codes with length < 5 with their correct zip code

In [16]:
masterDf['ZIP'] = masterDf['ZIP'].replace('', 'nan')

#replacing zip codes with len<5 with a nan values to they can be filled later
masterDf['ZIP'] = masterDf['ZIP'].apply(lambda x: x if len(str(x)) == 5 else 'nan')

### Filling null Zip Codes

In [17]:
print('Number of null zip code values before filling:', masterDf['ZIP'].isin(['nan']).sum())

Number of null zip code values before filling: 168574


In [18]:
#Loading GeoJSON of NC zip codes from GitHub

with urlopen("https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/nc_north_carolina_zip_codes_geo.min.json") as response:
    zip_codes = json.load(response)

#filtering json to only include charlotte zip codes that will be used in choropleth. GeoJSON files are so big that this makes massive performance increases.
filtered_features = [feature for feature in zip_codes['features'] if feature['properties']['ZCTA5CE10'] in clt_zip_codes]

# Creates a new GeoJSON object with filtered features
filtered_zipJSON = {
    "type": "FeatureCollection",
    "features": filtered_features
}

# Save filtered GeoJSON to file
with open('charlotte_zip_json', 'w') as f:
    json.dump(filtered_zipJSON, f)

#opening new filtered GeoJSON for later use
with open('charlotte_zip_json', 'r') as f:
    zip_codes = json.load(f)

#### Making a dictionary where the key is the zipcode and the value is a list of coordinate points that define a charlotte zipcode. Making this dictionary will allow me to later use coordiantes to find out what zip code a crime occoured in. The GeoJSON file has coorinate points in nested lists, but I need the polygon coordinate points to be in a 1 dimensional array for the shapely function I will later use

In [19]:

def flatten_coordinates(data):
    flattened_data = []
    
    def flatten_helper(sublist):
        for item in sublist:
            if isinstance(item, list):
                flatten_helper(item)
            elif isinstance(item, (int, float)):
                flattened_data.append(item)

    flatten_helper(data)
    tuples_data = [(flattened_data[i], flattened_data[i+1]) for i in range(0, len(flattened_data), 2)]
    return tuples_data

#initiating zip code dictionary
zip_coords_dict={}

for feature in zip_codes['features']:
    #setting the key equal to the zipcode
    zip_code = feature['properties']['ZCTA5CE10']

    zip_coordinates = flatten_coordinates(feature['geometry']['coordinates'])
    zip_coords_dict[zip_code]=zip_coordinates

#zip_coords_dict = {key: value for key, value in zip_coords_dict.items() if key in clt_zip_codes}

#### This function takes in a longitude and latitude as inputs and uses a shapely function (which utilizes ray casting algorithm) that returns True if a point is in a polygon and False if it is not. The function loops through all of the zipcodes until it gets a true value at which point the corresponding zip code is returned 

In [20]:
def zipCodeFromCoords(longitude, latitude):
    #if there is no latitude or longitude, the function does nothing
    if not longitude or not latitude:
        return "nan"
    elif longitude and latitude:
        
        point = Point(longitude, latitude)
        for zip_code_coords in list(zip_coords_dict.items()):
            z_code = zip_code_coords[0]
            #the coordinates of the zipcode are the values in dictionary
            poly = Polygon(zip_code_coords[1])
            
            if (poly.contains(point) == True):
                return z_code

### My first approach to filling null zip code values will be to use another dataset from data.charlottenc.gov that has information about 600,000+ addresses in charlotte. I can use the 'location' field in masterDf to look up the zip code in the address data set.

In [21]:

#Opening a charlotte dataset containing around 600,000 records of addresses, their zipcode, and other info. I wil be using this information
#to reduce the amount of NaN values in the dataset

AddressDF = pd.read_csv('Master_Address.csv')

In [22]:
#converting dataframe to dictionary where key is address and zip code is the value. dictionaries will be the most efficient data structure for the task of filling blank zip codes

AddressDF = AddressDF[['FullAddress', 'ZipCode']]
AddressDF.set_index('FullAddress', inplace=True)
AddressDict = AddressDF['ZipCode'].to_dict()

#function attempts to get the zip code attached the the address but if it cannot, it keeps the value as nan
def zipCodeGet(address):
    ZipCode = AddressDict.get(str(address))

    if ZipCode:
        return ZipCode
    else:
        return "nan"

In [23]:
#This block is responsible for filling all nan values of ZipCode for later use in the Choropleth map

# Define a function to process each row
def fill_from_database(row):
    if row['ZIP'] == 'nan':
        # Looking up the address in the database
        zip_from_location = str(zipCodeGet(row['LOCATION']))
        return zip_from_location
    else:
        return row['ZIP']

# Apply the function to each row of the DataFrame
masterDf['ZIP'] = masterDf.apply(fill_from_database, axis=1)

# Count the number of remaining null values
num_null_values = masterDf[masterDf['ZIP']=='nan'].shape[0]
print('Number of null values left after filling:', num_null_values)

Number of null values left after filling: 91555


### For the remaining null zip code values that could not be recovered from the address database, I will use their latitude and longitude to find out what zip code those coordiantes lie within. I will do this using the shapely library. (ZipCodeFromCoords() was defined earlier)

In [25]:
def process_row1(row):
    if row['ZIP'] != 'nan':
        return row['ZIP']
    elif pd.isna(row['LONGITUDE_PUBLIC']) or pd.isna(row['LATITUDE_PUBLIC']):
        return "nan"
    else:
        zip_code = str(zipCodeFromCoords(row['LONGITUDE_PUBLIC'], row['LATITUDE_PUBLIC']))
        return zip_code

# Apply the function to each row of the DataFrame
masterDf['ZIP'] = masterDf.apply(process_row1, axis=1)

# Count the number of remaining null values
num_null_values = masterDf[masterDf['ZIP']=='nan'].shape[0]
print('Number of null values left after filling:', num_null_values)

Number of null values left after filling: 0


In [26]:
#Finding out how long it takes CMPD to clear an incident

masterDf['TIME_TO_CLEAR'] = masterDf['CLEARANCE_DATE'] - masterDf['DATE_REPORTED']

print("The mean amount of days for CMPD to clear an incident is: {} days".format(masterDf['TIME_TO_CLEAR'].mean()))
masterDf['TIME_TO_CLEAR'].value_counts().head(10)

The mean amount of days for CMPD to clear an incident is: 50 days 13:38:59.008518194 days


TIME_TO_CLEAR
0 days    56162
1 days     9478
2 days     4805
6 days     4129
5 days     4038
7 days     4021
3 days     3958
4 days     3942
8 days     3273
9 days     2528
Name: count, dtype: int64

In [27]:
masterDf['TIME_TO_CLEAR'].isnull().value_counts()

TIME_TO_CLEAR
True     370375
False    166115
Name: count, dtype: int64

In [28]:
masterDf['ZIP'] = masterDf['ZIP'].astype(str)
zipCodeDF = masterDf[masterDf['ZIP']!='nan']


### Adding a column with the month in a number format and a column with an abbreviated month for use in dashboard

In [29]:
# Add 'MONTH' column with month numbers (1-12)
masterDf['MONTH'] = masterDf['DATE_REPORTED'].dt.month

# Add 'MONTH_NAME' column with month names
masterDf['MONTH_NAME'] = masterDf['DATE_REPORTED'].dt.strftime('%b')

month_order = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split(" ")
masterDf['MONTH_NAME'] = pd.Categorical(masterDf['MONTH_NAME'], categories=month_order, ordered=True)

**Adding day of the Week Column**

In [30]:
#Making a new column called DAY_WEEK indicating what day of the week a certain crime occoured. 0-Monday, 1-Tuesday... 6- Sunday this is for later use in a pie chart

def day_of_week(num):
    if num == 0:
        return 'Monday'
    elif num == 1:
        return 'Tuesday'
    elif num == 2:
        return 'Wednesday'
    elif num == 3:
        return 'Thursday'
    elif num == 4:
        return 'Friday'
    elif num == 5:
        return 'Saturday'
    elif num == 6:
        return 'Sunday'

masterDf['DAY_WEEK'] = masterDf['DATE_REPORTED'].apply(lambda x: x.dayofweek)
masterDf['DAY_WEEK'] = masterDf['DAY_WEEK'].apply(day_of_week)

#making sure only days of the week are in masterDf['DAY_WEEK']
masterDf['DAY_WEEK'].unique()

array(['Saturday', 'Thursday', 'Friday', 'Monday', 'Wednesday', 'Tuesday',
       'Sunday'], dtype=object)

In [33]:
# Save the cleaned DataFrame to a CSV file for use in the dashboard
masterDf.to_csv('CMPD_cleaned.csv') 